In [11]:
import openai, json

client = openai.OpenAI()
messages = []

In [3]:
def get_weather(city):
    return "33 degress celcius."

FUNCTION_MAP = {"get_weather": get_weather}

In [4]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties":{
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",   
                    },
                },
                "required": ["city"],
            },
        },
    },
]

In [ ]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    } 
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            
            print(f"Calling function: {function_name} with : {arguments}")
            
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
                
            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Run {function_name} with args{arguments} for a result of {result}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })
        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [8]:
while True:
    message = input("Send a message to the LLM...") 
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user","content": message})
        print(f"User: {message}")
        call_ai()

User: My name is Noori
AI: Hello Noori! How can I assist you today?
ChatCompletion(id='chatcmpl-D09BKiXjhlyOAsXMcTSuSsfrKs1ri', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello Noori! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768927666, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_c4585b5b9c', usage=CompletionUsage(completion_tokens=12, prompt_tokens=73, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
AI: Hello Noori! How can I assist you today?
User: what is my name
AI: Your name is Noori.
ChatCompletion(id='chatcmpl-D09BPKcgKYihQPRrjuN8c9zV8AdHR', choices=[Choice(finish_reason='stop'

In [10]:
messages

[{'role': 'user', 'content': 'my name is Noori'},
 {'role': 'user', 'content': 'My name is Noori'},
 {'role': 'assistant', 'content': 'Hello Noori! How can I assist you today?'},
 {'role': 'assistant', 'content': 'Hello Noori! How can I assist you today?'},
 {'role': 'user', 'content': 'what is my name'},
 {'role': 'assistant', 'content': 'Your name is Noori.'},
 {'role': 'assistant', 'content': 'Your name is Noori.'}]